In [38]:
import nltk

In [39]:
sents = ['Вася читает мою книгу', 'Напиши какое-нибудь письмо',
         'Этот веселый мальчик идет', 'Он любит читать всякие книги']

In [40]:
rules = """
    S -> NP VP | NP V | VP
    NP -> ADJ NP | ADJ N | NN | N
    VP -> V NP | V VP | V
    NN -> 'Вася'
    N -> 'книгу' | 'письмо' | 'мальчик' | 'Он' | 'книги'
    V -> 'читает' | 'Напиши' | 'идет' | 'любит' | 'читать'
    ADJ -> 'мою' | 'какое-нибудь' | 'Этот' | 'веселый' | 'всякие'
""".split('\n')

In [41]:
grammar = nltk.CFG.fromstring('\n'.join(rules))

In [42]:
def print_parses(parser, sentence):
    for elem in parser.parse(sentence.split()):
        print(elem)

# Алгоритм CYK

S | | | 
--- | ---
  | VP | | 
NP | | NP | 
NN | V | ADJ | N 
Вася | читает | мою | книгу 

Сработавшие правила:  
NN -> 'Вася'   
V -> 'читает'   
ADJ -> 'мою'   
N -> 'книгу'     
NP -> NN  
NP -> ADJ N   
VP -> V NP   
S -> NP VP   

# Алгоритм Эрли

In [43]:
cp_verbose = nltk.EarleyChartParser(grammar, trace=1)
print_parses(cp_verbose, "Вася читает мою книгу")

|.   Вася  .  читает .   мою   .  книгу  .|
|[---------]         .         .         .| [0:1] 'Вася'
|.         [---------]         .         .| [1:2] 'читает'
|.         .         [---------]         .| [2:3] 'мою'
|.         .         .         [---------]| [3:4] 'книгу'
|>         .         .         .         .| [0:0] S  -> * NP VP
|>         .         .         .         .| [0:0] S  -> * NP V
|>         .         .         .         .| [0:0] S  -> * VP
|>         .         .         .         .| [0:0] VP -> * V NP
|>         .         .         .         .| [0:0] VP -> * V VP
|>         .         .         .         .| [0:0] VP -> * V
|>         .         .         .         .| [0:0] NP -> * ADJ NP
|>         .         .         .         .| [0:0] NP -> * ADJ N
|>         .         .         .         .| [0:0] NP -> * NN
|>         .         .         .         .| [0:0] NP -> * N
|>         .         .         .         .| [0:0] NN -> * 'Вася'
|[---------]         .         .     

# Проверка

In [44]:
cp = nltk.EarleyChartParser(grammar)
for sent in sents:
    print('\n' + sent)
    print_parses(cp, sent)


Вася читает мою книгу
(S (NP (NN Вася)) (VP (V читает) (NP (ADJ мою) (NP (N книгу)))))
(S (NP (NN Вася)) (VP (V читает) (NP (ADJ мою) (N книгу))))

Напиши какое-нибудь письмо
(S (VP (V Напиши) (NP (ADJ какое-нибудь) (NP (N письмо)))))
(S (VP (V Напиши) (NP (ADJ какое-нибудь) (N письмо))))

Этот веселый мальчик идет
(S (NP (ADJ Этот) (NP (ADJ веселый) (NP (N мальчик)))) (V идет))
(S (NP (ADJ Этот) (NP (ADJ веселый) (N мальчик))) (V идет))
(S (NP (ADJ Этот) (NP (ADJ веселый) (NP (N мальчик)))) (VP (V идет)))
(S (NP (ADJ Этот) (NP (ADJ веселый) (N мальчик))) (VP (V идет)))

Он любит читать всякие книги
(S
  (NP (N Он))
  (VP (V любит) (VP (V читать) (NP (ADJ всякие) (NP (N книги))))))
(S
  (NP (N Он))
  (VP (V любит) (VP (V читать) (NP (ADJ всякие) (N книги)))))


# Улучшения

Внедрение в работу парсера морфологического анализатора. Это поможет 1) с определением частей речи и созданием новых правил для новых слов, 2) с приведением слова к начальной форме и записи правил только для нее.

# Неоднозначный разбор

**неоднозначность == омонимия**

Например, "Они увидели столовую(,) комнату." У слова "столовая" будет два разбора.

In [64]:
rules_ = """
    S -> NP VP 
    NP -> ADJ N | N N | NN
    VP -> V NP
    ADJ -> 'столовую'
    N -> 'столовую' | 'комнату' 
    V -> 'увидели' 
    NN -> 'Они'
""".split('\n')

In [65]:
grammar_ = nltk.CFG.fromstring('\n'.join(rules_))

In [66]:
cp_ = nltk.EarleyChartParser(grammar_)
sent = 'Они увидели столовую комнату'
print_parses(cp_, sent)

(S (NP (NN Они)) (VP (V увидели) (NP (N столовую) (N комнату))))
(S (NP (NN Они)) (VP (V увидели) (NP (ADJ столовую) (N комнату))))
